Code by Andrea Moscatello

Github: https://github.com/AndreaMoscatello/ChessAutoGUI/

In [1]:
import time
import pyautogui
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import CalibrationParameters as param
# reload the CalibrationParameters module
import importlib
importlib.reload(param)

# ML
import joblib # to save the model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Dense
from keras.models import load_model

# chess
from chessdotcom import *
import chess
import chess.engine
import chess.pgn

In [2]:
# printe the variable color_ratio from the CalibrationParameters.py file
print(param.x)

583


# Functions

In [17]:
# GENERATE fen string
FEN = {
    "white": {
        "pawn": "P",
        "knight": "N",
        "bishop": "B",
        "rook": "R",
        "queen": "Q",
        "king": "K"
    },
    "black": {
        "pawn": "p",
        "knight": "n",
        "bishop": "b",
        "rook": "r",
        "queen": "q",
        "king": "k"
    }
}

pieces = ['pawn', 'knight', 'bishop', 'rook', 'queen', 'king']

# check if a piece is black or white
def rgb_to_grayscale(rgb_image):
    return np.dot(rgb_image[..., :3], [0.2989, 0.5870, 0.1140])

def binarize_tile(tile, threshold=param.threshold):
    tile_array = np.uint8( tile[0] * 255 )
    # resize the tile by removing the borders (6 pixels on each side)
    tile_array = tile_array[8:-8, 8:-8, :]
    # convert the tile to grayscale
    gray_tile = rgb_to_grayscale(tile_array).astype(np.uint8)
    # apply thresholding to the grayscale image
    binary_tile = gray_tile > threshold
    binary_tile = binary_tile.astype(np.uint8) * 255
    return binary_tile

def GetPieceColor(tile):
    """
    tile should be from tiles_array
    """
    # convert the tile to a binary image
    binary_tile = binarize_tile(tile)

    # count the number of white pixels
    white_pixels = np.sum(binary_tile == 255)   
    black_pixels = np.sum(binary_tile == 0)
    ratio = white_pixels / black_pixels

    pieceColor = ""
    if ratio > param.color_ratio:
        pieceColor = "white"
    else:
        pieceColor = "black"

    return pieceColor

def get_FEN_2(tiles_array, model, encoder, FEN=FEN):
    FEN_string = ""
    for tile in tiles_array:
        # binarize the tile
        bw_tile = binarize_tile(tile, threshold=param.threshold)
        # if all pixel in the tile are the same it is an empty tile
        unique_values = np.unique(bw_tile)
        if len(unique_values) == 1:
            FEN_string += "1"
        else:
            # reshape the the tile to be compatible with the model
            bw_tile = np.pad(bw_tile, ((8, 8), (8, 8)), mode='constant', constant_values=bw_tile[0, 0])
            bw_tile = np.stack((bw_tile,)*3, axis=-1)
            bw_tile = bw_tile.reshape(1, 85, 85, 3)
            bw_tile = bw_tile / 255.0
            # evaluate the tile
            prediction = model.predict(bw_tile, verbose=0)
            prediction = encoder.inverse_transform(prediction)[0][0]
            # get the color of the piece
            pieceColor = GetPieceColor(bw_tile)
            # FEN character
            FEN_string += FEN[pieceColor][prediction]
    # add the '/' separator between every 8 charaters
    FEN_string = '/'.join(FEN_string[i:i+8] for i in range(0, len(FEN_string), 8))
    # sum all the consucutive numbers inside the string
    FEN_string = FEN_string.replace('11111111', '8')
    FEN_string = FEN_string.replace('1111111', '7')
    FEN_string = FEN_string.replace('111111', '6')
    FEN_string = FEN_string.replace('11111', '5')
    FEN_string = FEN_string.replace('1111', '4')
    FEN_string = FEN_string.replace('111', '3')
    FEN_string = FEN_string.replace('11', '2')
    return FEN_string

def get_screenshot(region=(param.x, param.y, param.width, param.width), tile_size=param.tile_size):
    """
    Capture a screenshot of a specified region and split it into tiles.
    
    Args:
        region (tuple): The region of the screen to capture the screenshot from. 
                        The region is defined by a tuple of (x, y, width, height).
                        Default is (583, 220, 672, 672). [all in px]
        tile_size (int): The size of each tile after resizing. Default is 85. [px]
    
    Returns:
        list: A list of numpy arrays representing the tiles of the screenshot.
    """
    
    x = region[0]
    y = region[1]
    width = region[2]
    heigth = region[3]

    im = pyautogui.screenshot(region=region)
    im.save('screenshot.png')  # save the screenshot to a file

    # split the image into 64 tiles
    tile_width = width // param.number_of_tiles
    tile_height = heigth // param.number_of_tiles

    tiles = []
    tiles_array = []
    # fill an array of tiles
    for i in range(param.number_of_tiles):
        for j in range(param.number_of_tiles):
            tile = im.crop((j * tile_width, i * tile_height, (j + 1) * tile_width, (i + 1) * tile_height))
            tile = tile.resize((tile_size, tile_size))
            tiles.append(tile)
            #tile.save(f'tile_{i}_{j}_resized.png')

            # convert the tile to a numpy array
            tile = tile.resize((tile_size, tile_size))
            tile = np.array(tile)
            tile = tile.reshape(1, tile_size, tile_size, 3)
            tile = tile / 255.0
            tiles_array.append(tile)

    return tiles_array

def EngineEvalution(game_info, user, engine, depth): 
    # extract moves from pgn
    pgn = game_info["pgn"]
    game = chess.pgn.read_game(io.StringIO(pgn)) # parse the PGN string
    moves = game.mainline_moves()
    board = game.board() # initialize board

    evaluation = []
    for move in moves:
        board.push(move) # make move
        fen = board.fen() # get fen from board
        info = engine.analyse(board, chess.engine.Limit(depth=param.depth)) # evaluate with engine

        # get eval from user side
        if game_info["white"]["username"] == user:
            ev = info["score"].white().score() # get relative evalutation in Cp
        else:
            ev = info["score"].black().score() # get relative evalutation in Cp
        evaluation.append( ev )
        
    return evaluation, info

In [34]:
# in the square defined by x, y, width, height find the centers of each square of size 85x85 pixels
def get_centers(x=param.x, y=param.y, width=param.width, heigth=param.heigth, tile_size=param.tile_size, color="white"):
    if color=="white":
        centers = []
        for i in range(param.number_of_tiles):
            for j in range(param.number_of_tiles):
                center = (x + j * tile_size + tile_size // 2, y + i * tile_size + tile_size // 2)
                centers.append(center)
        # flip the order of the centers
        centers = centers[::-1]
        # divide centers in batch of 8 and flip each odd batch
        centers = np.array(centers).reshape(param.number_of_tiles, param.number_of_tiles, 2)
        for i in range(param.number_of_tiles):
            centers[i] = centers[i][::-1]
        # flatten the array
        centers = centers.flatten().tolist()
        # couple the centers in pairs
        centers = [(centers[i], centers[i+1]) for i in range(0, len(centers), 2)]
    return centers

def make_move(best_move, color="white"):
    centers = get_centers()
    start = centers[best_move.from_square]
    end  = centers[best_move.to_square]
    # make the move
    pyautogui.moveTo(start)
    pyautogui.mouseDown(button='left')
    pyautogui.moveTo(end)  
    pyautogui.mouseUp()
    # move the mouse to the center-left of the screen to avoid overlaying the chees pieces during evaluation
    pyautogui.moveTo(0, param.heigth//2)
    

In [5]:
# define a FEN class to store the FEN string and its pieces: board position, player to move, castling rights
class FEN:
    def __init__(self, fen_string):
        self.fen_string = fen_string
        self.board_position = None
        self.player_to_move = None
        self.castling_rights = None
        self.en_passant_target = None
        self.halfmove_clock = None
        self.fullmove_number = None
        self.parse_fen_string(fen_string)

    def parse_fen_string(self, fen_string):
        parts = fen_string.split()
        self.board_position = parts[0]
        self.player_to_move = parts[1]
        self.castling_rights = parts[2]
        self.en_passant_target = parts[3]
        self.halfmove_clock = int(parts[4])
        self.fullmove_number = int(parts[5])

    def get_board_position(self):
        return self.board_position

    def get_player_to_move(self):
        return self.player_to_move

    def get_castling_rights(self):
        return self.castling_rights

    def get_en_passant_target(self):
        return self.en_passant_target

    def get_halfmove_clock(self):
        return self.halfmove_clock

    def get_fullmove_number(self):
        return self.fullmove_number

In [6]:
# Example usage
fen_string = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
fen = FEN(fen_string)

# Access individual components
print("Board Position:",    fen.get_board_position())
print("Player to Move:",    fen.get_player_to_move())
print("Castling Rights:",   fen.get_castling_rights())
print("En Passant Target:", fen.get_en_passant_target())
print("Halfmove Clock:",    fen.get_halfmove_clock())
print("Fullmove Number:",   fen.get_fullmove_number())

Board Position: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Player to Move: w
Castling Rights: KQkq
En Passant Target: -
Halfmove Clock: 0
Fullmove Number: 1


# Load Model and Engine

In [41]:
# load the encoder and model
encoder = joblib.load("encoder.pkl")
model   = load_model("model.keras")
print("Model loaded")

# load stockfish engine
stockfish_path = r"C:\Users\andre\Desktop\Programmazione\projects\chess\Stockfish 15\stockfish_15_win_x64_avx2\stockfish_15_x64_avx2.exe"
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
print("Engine loaded")

# test if the engine is working
board = chess.Board()
info = engine.analyse(board, chess.engine.Limit(time=0.1))
print(info["score"])


Model loaded
Engine loaded
PovScore(Cp(+38), WHITE)


# Run ChessAutoGUI

In [48]:
board = chess.Board() # initialize board
startingFEN = board.fen() # get the FEN string of the initial position
lastFEN = startingFEN

time.sleep(5)

lastBestMove = None

# every 5 seconds take a screenshot and get the FEN string
while True:
    try:
        tiles_array = get_screenshot()
        FEN_string  = get_FEN_2(tiles_array, model=model, encoder=encoder)
        print(FEN_string)
        
        if FEN_string != lastFEN or FEN_string == startingFEN:
            # from the FEN_string get the stockfish best move
            board.set_fen(FEN_string)
            
            # check if the position is checkmate
            if board.is_checkmate():
                print("Checkmate")
                break

            info = engine.analyse(board, chess.engine.Limit(depth=param.depth))
            best_move = info["pv"][0]
            print("Best move:", best_move)

            # get the evaluation
            evaluation = info["score"].white().score()
            print("Evaluation:", evaluation)
            print("\n")
            
            # make the move        
            if best_move != lastBestMove:
                make_move(best_move)
                lastBestMove = best_move

        time.sleep(3)
        
    except chess.engine.EngineTerminatedError:
        print("Engine died, reload the engine.")
        
        print("loading again the engine...")
        engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
        print("Engine loaded")
        
        # retry the above while loop
        continue
    

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
Best move: e2e4
Evaluation: 16


rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR
Best move: e4e5
Evaluation: 83


rnbqkb1r/pppppppp/8/4P3/4n3/8/PPPP1PPP/RNBQKBNR
Best move: d2d4
Evaluation: 123


rnbqkb1r/pppp1ppp/4p3/4P3/3Pn3/8/PPP2PPP/RNBQKBNR
Best move: g1h3
Evaluation: 221


rnbqkb1r/ppp2ppp/4p3/3pP3/3Pn3/7N/PPP2PPP/RNBQKB1R
Best move: f2f3
Evaluation: 548


rnbqk2r/ppp1bppp/4p3/3pP3/3Pn3/5P1N/PPP3PP/RNBQKB1R
Best move: f3e4
Evaluation: 578


rnbqk2r/pp2bppp/4p3/2ppP3/3PP3/7N/PPP3PP/RNBQKB1R
Best move: d1g4
Evaluation: 503


rnbqk2r/pp2bppp/4p3/2p1P3/3Pp1Q1/7N/PPP3PP/RNB1KB1R
Best move: d4c5
Evaluation: 497


rnbqk2r/pp2bppp/4p3/2P1P3/4p1Q1/7N/PPP3PP/RNB1KB1R
Best move: c1e3
Evaluation: 779


rnbq1rk1/pp2bppp/4p3/2P1P3/4p1Q1/4B2N/PPP3PP/RN2KB1R
Best move: e3h6
Evaluation: 711


r1bq1rk1/pp2bp1p/n3p2B/2P1P1p1/4p1Q1/7N/PPP3PP/RN2KB1R
Best move: h3g5
Evaluation: 980


r1bq1rk1/pp3p1p/n3p2B/2P1P1b1/4p1Q1/8/PPP3PP/RN2KB1R
Best move: h6g5
Evalu

KeyboardInterrupt: 